Script em Python que lê os dados de uma tabela num banco MySQL, consolida os dados e salva numa outra tabela de um banco local MySQL.

In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import mysql.connector as mysql
import pandas as pd

# Conexão ao banco de dados remoto
con_remote = mysql.connect(
    host="40b8f30251.nxcli.io",
    port=3306,
    user="a4f2b49a_padawan",
    password="KaratFlanksUgliedSpinal",
    database="a4f2b49a_sample_database",
)

# Conexão ao banco de dados local
con_local = mysql.connect(
    host="localhost",
    port=3306,
    user="root",
    password="tom*1005",
    database="H2_schema", auth_plugin='mysql_native_password'
)

# Leitura dos dados da tabela "raw_data"
query = "SELECT * FROM raw_data"
df = pd.read_sql(query, con=con_remote)

# Consolidação dos dados por mês
df['datahora_acesso'] = df['datahora_acesso'].str[:10]  # Pega apenas os primeiros 10 caracteres (YYYY-MM-DD)
df['mes'] = pd.to_datetime(df['datahora_acesso'], format='%Y-%m-%d').dt.month

def filter_cash_game(df_group):
    return df_group[df_group['modalidade'] == 'Cash Game']

def filter_torneio(df_group):
    return df_group[df_group['modalidade'] == 'Torneio']

cash_game_filtered = filter_cash_game(df)
torneio_filtered = filter_torneio(df)

df_consolidado = df.groupby('mes').agg(
    rake=pd.NamedAgg(column='rake', aggfunc='sum'),
    jogadores=pd.NamedAgg(column='clientes_id', aggfunc='nunique'),
    rake_cash_game=pd.NamedAgg(column='rake', aggfunc=lambda x: cash_game_filtered['rake'].sum()),  # Use filtered column directly
    rake_torneio=pd.NamedAgg(column='rake', aggfunc=lambda x: torneio_filtered['rake'].sum()),  # Use filtered column directly
    jogadores_cash_game=pd.NamedAgg(column='clientes_id', aggfunc='nunique'),  # No data argument needed
    jogadores_torneio=pd.NamedAgg(column='clientes_id', aggfunc='nunique'),  # No data argument needed
)





# Criação da tabela "consolidado" no banco local
cursor = con_local.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS consolidado (
        mes INT,
        rake DECIMAL(10,2),
        jogadores INT,
        rake_cash_game DECIMAL(10,2),
        rake_torneio DECIMAL(10,2),
        jogadores_cash_game INT,
        jogadores_torneio INT
    )
""")

# Gravação dos dados consolidados no banco local
for row in df_consolidado.to_dict('records'):
    cursor.execute("""
        INSERT INTO consolidado (rake, jogadores, rake_cash_game, rake_torneio, jogadores_cash_game, jogadores_torneio)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (row['rake'], row['jogadores'], row['rake_cash_game'], row['rake_torneio'], row['jogadores_cash_game'], row['jogadores_torneio']))

con_local.commit()

# Fechamento das conexões
con_remote.close()
con_local.close()


/var/folders/n_/sns0jy897853mnl32jkbqpdm0000gn/T/ipykernel_5918/1954775806.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=con_remote)
